In [3]:
import pandas as pd
import os
from tabulate import tabulate

# Load dateset
df = pd.read_csv(os.getcwd() + '/../../data/edge-iiot/Edge-IIoTset dataset/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.csv', low_memory=False)

# Split dataset to attack and normal
attack_df = df[df['Attack_label'] == 1]
normal_df = df[df['Attack_label'] == 0]

# Drop columns
attack_df = attack_df.drop(columns=['Attack_label', 'Attack_type'])
normal_df = normal_df.drop(columns=['Attack_label', 'Attack_type'])

# Split dataset into training and test set
attack_df_train = attack_df.sample(frac=0.8, random_state=42)
attack_df_test = attack_df.drop(attack_df_train.index)
normal_df_train = normal_df.sample(frac=0.8, random_state=42)
normal_df_test = normal_df.drop(normal_df_train.index)

# Print dataset sizes in a table
data = [
    ["Attack", attack_df.shape[0], attack_df_train.shape[0], attack_df_test.shape[0]],
    ["Normal", normal_df.shape[0], normal_df_train.shape[0], normal_df_test.shape[0]]
]
print(tabulate(data, headers=["Atack type", "Total", "Train", "Test"], tablefmt="grid"))

+--------------+---------+---------+--------+
| Atack type   |   Total |   Train |   Test |
+==============+=========+=========+========+
| Attack       |  133499 |  106799 |  26700 |
+--------------+---------+---------+--------+
| Normal       |   24301 |   19441 |   4860 |
+--------------+---------+---------+--------+


In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory="./chroma_db_binary")
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 5})

/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
from statistics import mode
from sklearn.metrics import classification_report
from tqdm import tqdm

sample_size = 10 # attack_df_test.shape[0]

datasets = {
    "attack": attack_df_test,
    "normal": normal_df_test
}
y_pred = []
y_true = []

for attack_type, dataset in datasets.items():
    sample_size = dataset.shape[0]
    for i in tqdm(range(sample_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        query_document = str(dataset.iloc[i].to_list())
        similar_documents = retriever.invoke(query_document, filter={"source": "edge-iiotset"})
        y_true.append(attack_type)
        y_pred.append(mode([doc.metadata["label"] for doc in similar_documents]))

print(classification_report(y_true, y_pred))

Predicting normal entries...: 100%|█████████████████████████████| 4860/4860 [31:37<00:00,  2.56it/s]


              precision    recall  f1-score   support

      attack       1.00      0.99      0.99     26700
      normal       0.94      1.00      0.97      4860

    accuracy                           0.99     31560
   macro avg       0.97      0.99      0.98     31560
weighted avg       0.99      0.99      0.99     31560



In [11]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

array([[26389,   311],
       [   24,  4836]])

In [7]:
# save y_true and y_pred to a file
with open("y_true.txt", "w") as f:
    for item in y_true:
        f.write("%s\n" % item)

with open("y_pred.txt", "w") as f:
    for item in y_pred:
        f.write("%s\n" % item)